In [1]:
import json
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import ViTForImageClassification, ViTFeatureExtractor, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import cv2

# Load annotations
def load_annotations(json_file):
    with open(json_file, 'r') as f:
        annotation_data = json.load(f)
    image_id_to_file = {img['id']: img['file_name'] for img in annotation_data['images']}
    annotations = {image_id_to_file[ann['image_id']]: ann['category_id'] for ann in annotation_data['annotations']}
    return annotations

# Data generator function
def data_generator(directory, annotations, batch_size=32, target_size=(224, 224)):
    files = list(annotations.keys())
    num_files = len(files)
    while True:
        np.random.shuffle(files)
        for offset in range(0, num_files, batch_size):
            batch_files = files[offset:offset+batch_size]
            images = []
            labels = []
            for file in batch_files:
                image_path = os.path.join(directory, file)
                image = cv2.imread(image_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, target_size)
                images.append(image)
                labels.append(annotations[file])
            yield np.array(images) / 255.0, np.array(labels)

# Custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, directory, annotations, transform=None):
        self.directory = directory
        self.annotations = annotations
        self.transform = transform
        self.files = list(annotations.keys())

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file = self.files[idx]
        image_path = os.path.join(self.directory, file)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224))
        label = self.annotations[file]

        if self.transform:
            image = self.transform(image)

        return {"pixel_values": torch.tensor(image).permute(2, 0, 1).float(), "label": torch.tensor(label).long()}

# Load annotations
train_annotations = load_annotations(r'dataset\train\_annotations.coco.json')
val_annotations = load_annotations(r'dataset\valid\_annotations.coco.json')

# Create datasets
train_dataset = CustomDataset(r'dataset\train', train_annotations)
val_dataset = CustomDataset(r'dataset\valid', val_annotations)

# Load the pre-trained ViT model and feature extractor
model_name = "google/vit-base-patch16-224"
num_classes = 4

# Ignore size mismatches for the classifier layer
model = ViTForImageClassification.from_pretrained(model_name, num_labels=num_classes, ignore_mismatched_sizes=True)
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)

# Define compute metrics function
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

# Use mixed precision training
training_args = TrainingArguments(
    per_device_train_batch_size=16,  # Increase batch size for faster training
    evaluation_strategy="steps",
    num_train_epochs=10,  # Increase number of epochs
    learning_rate=5e-5,  # Adjust learning rate
    weight_decay=0.01,
    save_steps=500,
    save_total_limit=2,
    output_dir="./vit-finetuned",
    logging_dir="./vit-finetuned/logs",
    fp16=True,  # Enable mixed precision training
    logging_steps=10,
    eval_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Save the fine-tuned model
trainer.save_model("./vit-finetuned")


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Swakshwar\AppData\Roaming\Python\Python310\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


  0%|          | 0/2360 [00:00<?, ?it/s]

{'loss': 1.3849, 'grad_norm': 4.339969635009766, 'learning_rate': 4.9830508474576276e-05, 'epoch': 0.04}
{'loss': 1.2616, 'grad_norm': 3.3621714115142822, 'learning_rate': 4.961864406779661e-05, 'epoch': 0.08}
{'loss': 1.2247, 'grad_norm': 5.570422172546387, 'learning_rate': 4.940677966101695e-05, 'epoch': 0.13}
{'loss': 1.0638, 'grad_norm': 4.337039947509766, 'learning_rate': 4.919491525423729e-05, 'epoch': 0.17}
{'loss': 1.1508, 'grad_norm': 6.269214630126953, 'learning_rate': 4.898305084745763e-05, 'epoch': 0.21}
{'loss': 1.036, 'grad_norm': 6.752583980560303, 'learning_rate': 4.877118644067797e-05, 'epoch': 0.25}
{'loss': 1.0256, 'grad_norm': 4.5840678215026855, 'learning_rate': 4.855932203389831e-05, 'epoch': 0.3}
{'loss': 0.9327, 'grad_norm': nan, 'learning_rate': 4.8368644067796615e-05, 'epoch': 0.34}
{'loss': 1.0286, 'grad_norm': 6.176445960998535, 'learning_rate': 4.815677966101695e-05, 'epoch': 0.38}
{'loss': 1.0096, 'grad_norm': 5.513400077819824, 'learning_rate': 4.79449152

  0%|          | 0/114 [00:00<?, ?it/s]

{'eval_loss': 1.2564431428909302, 'eval_accuracy': 0.4718232044198895, 'eval_precision': 0.5423971270575607, 'eval_recall': 0.4718232044198895, 'eval_f1': 0.4314397442986558, 'eval_runtime': 15.3002, 'eval_samples_per_second': 59.15, 'eval_steps_per_second': 7.451, 'epoch': 0.42}
{'loss': 0.9487, 'grad_norm': 5.125683307647705, 'learning_rate': 4.7733050847457624e-05, 'epoch': 0.47}
{'loss': 0.9688, 'grad_norm': 5.3657755851745605, 'learning_rate': 4.752118644067797e-05, 'epoch': 0.51}
{'loss': 1.023, 'grad_norm': 10.060126304626465, 'learning_rate': 4.7309322033898304e-05, 'epoch': 0.55}
{'loss': 0.9265, 'grad_norm': 4.157229900360107, 'learning_rate': 4.709745762711865e-05, 'epoch': 0.59}
{'loss': 0.9891, 'grad_norm': 5.18560266494751, 'learning_rate': 4.6885593220338983e-05, 'epoch': 0.64}
{'loss': 0.9145, 'grad_norm': 6.376780033111572, 'learning_rate': 4.667372881355933e-05, 'epoch': 0.68}
{'loss': 0.9515, 'grad_norm': 7.651336193084717, 'learning_rate': 4.646186440677966e-05, 'ep

...

{'eval_loss': 1.6206227540969849, 'eval_accuracy': 0.6497237569060773, 'eval_precision': 0.6465021096900179, 'eval_recall': 0.6497237569060773, 'eval_f1': 0.6454800092068237, 'eval_runtime': 6.9526, 'eval_samples_per_second': 130.167, 'eval_steps_per_second': 16.397, 'epoch': 8.9}
{'loss': 0.0009, 'grad_norm': 0.0855693519115448, 'learning_rate': 5.360169491525424e-06, 'epoch': 8.94}
{'loss': 0.001, 'grad_norm': 0.10975849628448486, 'learning_rate': 5.148305084745763e-06, 'epoch': 8.98}
{'loss': 0.0009, 'grad_norm': 0.05465579405426979, 'learning_rate': 4.936440677966102e-06, 'epoch': 9.03}
{'loss': 0.001, 'grad_norm': 0.04077761247754097, 'learning_rate': 4.724576271186441e-06, 'epoch': 9.07}
{'loss': 0.0008, 'grad_norm': 0.027294278144836426, 'learning_rate': 4.51271186440678e-06, 'epoch': 9.11}
{'loss': 0.001, 'grad_norm': 0.047495290637016296, 'learning_rate': 4.300847457627119e-06, 'epoch': 9.15}
{'loss': 0.0008, 'grad_norm': 0.04055822268128395, 'learning_rate': 4.088983050847458

  0%|          | 0/114 [00:00<?, ?it/s]

{'eval_loss': 1.6298973560333252, 'eval_accuracy': 0.6541436464088398, 'eval_precision': 0.6509033363158168, 'eval_recall': 0.6541436464088398, 'eval_f1': 0.6497853352834244, 'eval_runtime': 6.9971, 'eval_samples_per_second': 129.338, 'eval_steps_per_second': 16.292, 'epoch': 9.32}
{'loss': 0.0008, 'grad_norm': 0.06421801447868347, 'learning_rate': 3.241525423728814e-06, 'epoch': 9.36}
{'loss': 0.0008, 'grad_norm': 0.04872427135705948, 'learning_rate': 3.029661016949153e-06, 'epoch': 9.41}
{'loss': 0.0008, 'grad_norm': 0.04480556398630142, 'learning_rate': 2.817796610169492e-06, 'epoch': 9.45}
{'loss': 0.0008, 'grad_norm': 0.08226297795772552, 'learning_rate': 2.6059322033898303e-06, 'epoch': 9.49}
{'loss': 0.0009, 'grad_norm': 0.036264531314373016, 'learning_rate': 2.3940677966101697e-06, 'epoch': 9.53}
{'loss': 0.0011, 'grad_norm': 0.0338827483355999, 'learning_rate': 2.1822033898305086e-06, 'epoch': 9.58}
{'loss': 0.0007, 'grad_norm': 0.05246584862470627, 'learning_rate': 1.97033898

  0%|          | 0/114 [00:00<?, ?it/s]

{'eval_loss': 1.6342209577560425, 'eval_accuracy': 0.6574585635359116, 'eval_precision': 0.6548113782584233, 'eval_recall': 0.6574585635359116, 'eval_f1': 0.6535494395842923, 'eval_runtime': 7.0274, 'eval_samples_per_second': 128.781, 'eval_steps_per_second': 16.222, 'epoch': 9.75}
{'loss': 0.0008, 'grad_norm': 0.022197166457772255, 'learning_rate': 1.1228813559322035e-06, 'epoch': 9.79}
{'loss': 0.0009, 'grad_norm': 0.0444762147963047, 'learning_rate': 9.110169491525425e-07, 'epoch': 9.83}
{'loss': 0.001, 'grad_norm': 0.03895612806081772, 'learning_rate': 6.991525423728814e-07, 'epoch': 9.87}
{'loss': 0.0009, 'grad_norm': 0.03454240411520004, 'learning_rate': 4.872881355932204e-07, 'epoch': 9.92}
{'loss': 0.0007, 'grad_norm': 0.06842169910669327, 'learning_rate': 2.7542372881355935e-07, 'epoch': 9.96}
{'loss': 0.0009, 'grad_norm': 0.1448354721069336, 'learning_rate': 6.355932203389831e-08, 'epoch': 10.0}
{'train_runtime': 978.5686, 'train_samples_per_second': 38.434, 'train_steps_per_

In [2]:
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

In [1]:
# Evaluate the model on the validation dataset
evaluation_results = trainer.evaluate()
print(f"Evaluation Results: {evaluation_results}")

Evaluation Results: {'eval_loss': 1.6354875564575195, 'eval_accuracy': 0.8552486187845303, 'eval_precision': 0.852748436391131, 'eval_recall': 0.8552486187845303, 'eval_f1': 0.8513106008106088, 'eval_runtime': 6.9583, 'eval_samples_per_second': 130.061, 'eval_steps_per_second': 16.383, 'epoch': 10.0}



In [4]:
# Generate predictions
predictions, labels, metrics = trainer.predict(val_dataset)
preds = np.argmax(predictions, axis=1)

  0%|          | 0/114 [00:00<?, ?it/s]

In [5]:
# Confusion Matrix
conf_matrix = confusion_matrix(labels, preds)
print('Confusion Matrix')
print(conf_matrix)

Confusion Matrix
[[187  13  35  20]
 [ 18 101  29  50]
 [ 30  40 110  29]
 [ 10  16  22 195]]


In [2]:
# Classification Report
class_report = classification_report(labels, preds, target_names=["Class 0", "Class 1", "Class 2", "Class 3"])
print('Classification Report')
print(class_report)


Classification Report
              precision    recall  f1-score   support

     Class 0       0.76      0.73      0.75       255
     Class 1       0.79      0.51      0.55       198
     Class 2       0.76      0.53      0.54       209
     Class 3       0.76      0.80      0.73       243

    accuracy                           0.86       905
   macro avg       0.78      0.84      0.84       905
weighted avg       0.78      0.86      0.85       905





In [7]:
# ROC Curve and AUC (for binary classification)
if num_classes == 2:
    fpr, tpr, _ = roc_curve(labels, predictions[:, 1])
    roc_auc = auc(fpr, tpr)
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc='lower right')
    plt.show()